# Kaggle Home Credit Default Risk Competition

https://www.kaggle.com/c/home-credit-default-risk

This competition is sponsored by [Home Credit](http://www.homecredit.net), whose mission is to provide a positive and safe borrowing experience to groups of people that traditional, mainstream banks and financial institutions typically refuse to serve.

In order to make lending decisions on applicants from this demographic, Home Credit needs an algorithm that will take as inputs various financial and personal information originally taken from a loan applicant's profile, and then compute a probability that the applicant will have trouble paying back the loan. This probability will be in the range [0.0, 1.0], where 1.0 represents a 100% certainty that the applicant will have repayment difficulties and 0.0 indicates that there is zero chance that the applicant will ever miss any payments. The algorithm will be tested and ranked on Kaggle based on a set predictions it makes for 48,744 individuals who previously borrowed from Home Credit.

Solution algorithms will be trained on a set of datapoints from 307,511 previous Home Credit borrowers. It is imperative that some portion, say 20%, of the training set is set aside to serve as a validation set. Alternatively, an algorithm such as K-Fold Cross Validation could be used.

To submit a solution on Kaggle, a CSV file must be produced that contains one header row, and 48,744 prediction rows, where each prediction row contains both a user ID, the `SKI_ID_CURR` column, and the probability, the `TARGET` column, of that user having repayment defficulties. The file must be formatted as follows:

    SK_ID_CURR,TARGET
    100001,0.1
    100005,0.9
    100013,0.2
    etc.
    
Home Credit knows which borrowers in the test set were delinquent, and which ones never made a late payment. A good algorithm will need to predict a high probability of delinquent repayment for the majority of borrowers who did in fact make late payments (those whose `TARGET` value is 1 in the main table in the dataset). This algorithm will also need to predict a low probability of delinquent repayment for the majority of borrowers who never made a late payment (those whose `TARGET` value is 0 in the main table in the dataset).

## I. Data Exploration

In [16]:
# Import libraries necessary for this project.
import numpy as np
import pandas as pd
from time import time
from IPython.display import display # Allows the use of display() for dataframes.

# Have all columns appear when dataframes are displayed.
pd.set_option('display.max_columns', None) 

# Have 100 rows appear when a dataframe is displayed
pd.set_option('display.max_rows', 500)

# Display dimensions whenever a dataframe is printed out.
pd.set_option('display.show_dimensions', True)

# Pretty display for notebooks.
%matplotlib inline 

### Data Description

From https://www.kaggle.com/c/home-credit-default-risk/data:

1. **application_{train|test}.csv**
   * This is the main table, broken into two files for Train (with TARGET) and Test (without TARGET).
   * Static data for all applications. One row represents one loan in our data sample.
<p>
<p>
* **bureau.csv**
   * All client's previous credits provided by other financial institutions that were reported to Credit Bureau (for clients who have a loan in our sample).
   * For every loan in our sample, there are as many rows as number of credits the client had in Credit Bureau before the application date.
<p>
<p>
* **bureau_balance.csv**

  * Monthly balances of previous credits in Credit Bureau.
  * This table has one row for each month of history of every previous credit reported to Credit Bureau – i.e the table has (#loans in sample * # of relative previous credits * # of months where we have some history observable for the previous credits) rows.
<p>
<p>
* **previous_application.csv**

  * All previous applications for Home Credit loans of clients who have loans in our sample.
  * There is one row for each previous application related to loans in our data sample.
<p>
<p>
* **POS_CASH_balance.csv**

  * Monthly balance snapshots of previous POS (point of sales) and cash loans that the applicant had with Home Credit.
  * This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credits * # of months in which we have some history observable for the previous credits) rows.
<p>
<p>
* **installments_payments.csv**

  * Repayment history for the previously disbursed credits in Home Credit related to the loans in our sample.
  * There is a) one row for every payment that was made plus b) one row each for missed payment.
  * One row is equivalent to one payment of one installment OR one installment corresponding to one payment of one previous Home Credit credit related to loans in our sample.
<p>
<p>
* **credit_card_balance.csv**

  * Monthly balance snapshots of previous credit cards that the applicant has with Home Credit.
  * This table has one row for each month of history of every previous credit in Home Credit (consumer credit and cash loans) related to loans in our sample – i.e. the table has (#loans in sample * # of relative previous credit cards * # of months where we have some history observable for the previous credit card) rows.

![Data Table Diagram](images/homecredit.png "Data Table Diagram")

In [2]:
# Load the data tables
application_train_data = pd.read_csv("data/application_train.csv")
application_test_data = pd.read_csv("data/application_test.csv")
bureau_data = pd.read_csv("data/bureau.csv")
bureau_balance_data = pd.read_csv("data/bureau_balance.csv")
previous_application_data = pd.read_csv("data/previous_application.csv")
POS_CASH_balance_data = pd.read_csv("data/POS_CASH_balance.csv")
installments_payments_data = pd.read_csv("data/installments_payments.csv")
credit_card_balance_data = pd.read_csv("data/credit_card_balance.csv")

### 1. Main Data Table (application_{train|test}.csv)

In [3]:
# Total number of entries in training group
print("Total number of entries in training group: {}".format(application_train_data.shape[0]))

Total number of entries in training group: 307511


In [4]:
# Total number of entries in test group
print("Total number of entries in test group: {}".format(application_test_data.shape[0]))

Total number of entries in test group: 48744


In [5]:
# Total number of features in the main (application) data table
print("Total number of features in main (application) data table: {}".format(application_train_data.shape[1]))

Total number of features in main (application) data table: 122


In [6]:
# Display the first five records
display(application_train_data.head(n=5))

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,ORGANIZATION_TYPE,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,Business Entity Type 3,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,0.1250,0.0369,0.0202,0.0190,0.0000,0.0000,0.0252,0.0383,0.9722,0.6341,0.0144,0.0000,0.0690,0.0833,0.1250,0.0377,0.022,0.0198,0.0,0.0,0.0250,0.0369,0.9722,0.6243,0.0144,0.00,0.0690,0.0833,0.1250,0.0375,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,School,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,0.3333,0.0130,0.0773,0.0549,0.0039,0.0098,0.0924,0.0538,0.9851,0.8040,0.0497,0.0806,0.0345,0.2917,0.3333,0.0128,0.079,0.0554,0.0,0.0,0.0968,0.0529,0.9851,0.7987,0.0608,0.08,0.0345,0.2917,0.3333,0.0132,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,Government,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

In [7]:
# Display a statistical description of the numerical features in the main (application) data table.
display(application_train_data.describe())

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,FLOORSMIN_AVG,LANDAREA_AVG,LIVINGAPARTMENTS_AVG,LIVINGAREA_AVG,NONLIVINGAPARTMENTS_AVG,NONLIVINGAREA_AVG,APARTMENTS_MODE,BASEMENTAREA_MODE,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_MODE,COMMONAREA_MODE,ELEVATORS_MODE,ENTRANCES_MODE,FLOORSMAX_MODE,FLOORSMIN_MODE,LANDAREA_MODE,LIVINGAPARTMENTS_MODE,LIVINGAREA_MODE,NONLIVINGAPARTMENTS_MODE,NONLIVINGAREA_MODE,APARTMENTS_MEDI,BASEMENTAREA_MEDI,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BUILD_MEDI,COMMONAREA_MEDI,ELEVATORS_MEDI,ENTRANCES_MEDI,FLOORSMAX_MEDI,FLOORSMIN_MEDI,LANDAREA_MEDI,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,TOTALAREA_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
count,307511.000000,307511.000000,307511.000000,3.075110e+05,3.075110e+05,307499.000000,3.072330e+05,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,104582.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307509.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,134133.000000,3.068510e+05,246546.000000,151450.00000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,151450.000000,127568.000000,157504.000000,103023.000000,92646.000000,143620.000000,152683.000000,154491.000000,98869.000000,124921.000000,97312.000000,153161.000000,93997.000000,137829.000000,159080.000000,306490.000000,306490.000000,306490.000000,306490.000000,307510.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.00000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,307511.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000,265992.000000
mean,278180.518577,0.080729,0.417052,1.687979e+05,5.990260e+05,27108.573909,5.383962e+05,0.020868,-16036.995067,63815.045904,-4986.120328,-2994.202373,12.061091,0.999997,0.819889,0.199368,0.998133,0.281066,0.056720,2.152665,2.052463,2.031521,12.063419,0.015144,0.050769,0.040659,0.078173,0.230454,0.179555,0.502130,5.143927e-01,0.510853,0.11744,0.088442,0.977735,0.752471,0.044621,0.078942,0.149725,0.226282,0.231894,0.066333,0.100775,0.107399,0.008809,0.028358,0.114231,0.087543,0.977065,0.759637,0.042553,0.074490,0.145193,0.222315,0.228058,0.064958,0.105645,0.105975,0.008076,0.027022,0.117850,0.087955,0.97

In [9]:
# Count number of delinquent repayers ('TARGET' value of 1) and non-delinquent repayers 
# ('TARGET' value of 0) in the training set of the main data table.

application_train_data['TARGET'].value_counts()

0    282686
1     24825
Name: TARGET, Length: 2, dtype: int64

In [10]:
# Fraction of applicants in training set who were delinquent repayers. If you took a random 
# sample, this is the probability you would select a delinquent repayer by chance:
repayers_fraction = round(24825/(24825+282686), 4)
print('Fraction of training set who were delinquent payers: {}'.format(repayers_fraction))

Fraction of training set who were delinquent payers: 0.0807


### Main Data Table 'NaN' entry investigation

In [21]:
# Get numerical counts of number of NaN entries in each column (feature) in the main data table.
features_sorted_by_NaN_count = application_train_data.isnull().sum().sort_values(ascending=False)

# Display only features with NaN counts greater than 0.
feature_NaN_counts = features_sorted_by_NaN_count[features_sorted_by_NaN_count > 0]

# See which, if any, features' NaN data might still be useful in predicting 
# delinquent repayers.
application_train_data[(application_train_data['COMMONAREA_MEDI'].isnull()) & (application_train_data['TARGET'] == 1)].shape

# Create a dataframe to summarize 'NaN' entries in the main data table (application_train_data)
NaN_summary = pd.DataFrame(index=feature_NaN_counts.index, columns=['#_NaN_Entries','Fraction_of_entries_that_are_NaN','#_NaN_entries_who_are_Delinquent','Fraction_of_NaN_entries_who_are_Delinquent','#non_NaN_Entries','#non_NaN_entries_who_are_Delinquent','Fraction_of_non_NaN_entries_who_are_Delinquent','Significant'])

# Fill each row in the NaN summary dataframe
for feature_name in NaN_summary.index:
    # Get the amount and fraction of delinquents among all borrowers who 
    # have an 'NaN' entry for a particular feature. Do this for each feature that has at least 
    # one 'NaN' entry.
    number_of_NaN = feature_NaN_counts.loc[feature_name]
    NaN_summary['#_NaN_Entries'][feature_name] = number_of_NaN
    number_delinquents_who_are_NaN = application_train_data[(application_train_data[feature_name].isnull()) & (application_train_data['TARGET'] == 1)].shape[0]
    NaN_summary['#_NaN_entries_who_are_Delinquent'][feature_name] = number_delinquents_who_are_NaN
    fraction_of_NaN_entries_who_are_delinquents = round(number_delinquents_who_are_NaN/number_of_NaN,4)
    NaN_summary['Fraction_of_NaN_entries_who_are_Delinquent'][feature_name] = fraction_of_NaN_entries_who_are_delinquents
    
    # Get the amount of non-'NaN' entries in each feature that 
    # has at least one 'NaN' entry.
    number_of_records = application_train_data[feature_name].shape[0]
    number_of_non_NaN = number_of_records - number_of_NaN
    NaN_summary['#non_NaN_Entries'][feature_name] = number_of_non_NaN
    
    # Get the fraction of the total entries for a feature that are 'NaN'
    fraction_of_feature_entries_that_are_NaN = round(number_of_NaN/(number_of_NaN+number_of_non_NaN),4)
    NaN_summary['Fraction_of_entries_that_are_NaN'][feature_name] = fraction_of_feature_entries_that_are_NaN
    
    # Get the amount and fraction of delinquents among all borrowers who 
    # have a non-'NaN' entry for a particular feature. Do this for each feature that has at least 
    # one 'NaN' entry.
    number_delinquents_who_are_not_NaN = application_train_data[(application_train_data[feature_name].notnull()) & (application_train_data['TARGET'] == 1)].shape[0]
    NaN_summary['#non_NaN_entries_who_are_Delinquent'][feature_name] = number_delinquents_who_are_not_NaN
    fraction_of_non_NaN_entries_who_are_delinquents = round(number_delinquents_who_are_not_NaN/number_of_non_NaN,4)
    NaN_summary['Fraction_of_non_NaN_entries_who_are_Delinquent'][feature_name] = fraction_of_NaN_entries_who_are_delinquents

# Display the NaN summary dataframe below
display(NaN_summary, 'display.max_columns')

,#_NaN_Entries,Fraction_of_entries_that_are_NaN,#_NaN_entries_who_are_Delinquent,Fraction_of_NaN_entries_who_are_Delinquent,#non_NaN_Entries,#non_NaN_entries_who_are_Delinquent,Fraction_of_non_NaN_entries_who_are_Delinquent,Significant
COMMONAREA_MEDI,214865,0.6987,18423,0.0857,92646,6402,0.0857,NaN
COMMONAREA_AVG,214865,0.6987,18423,0.0857,92646,6402,0.0857,NaN
COMMONAREA_MODE,214865,0.6987,18423,0.0857,92646,6402,0.0857,NaN
NONLIVINGAPARTMENTS_MODE,213514,0.6943,18327,0.0858,93997,6498,0.0858,NaN
NONLIVINGAPARTMENTS_MEDI,213514,0.6943,18327,0.0858,93997,6498,0.0858,NaN
NONLIVINGAPARTMENTS_AVG,213514,0.6943,18327,0.0858,93997,6498,0.0858,NaN
FONDKAPREMONT_MODE,210295,0.6839,18125,0.0862,97216,6700,0.0862,NaN
LIVINGAPARTMENTS_MEDI,210199,0.6835,18122,0.0862,97312,6703,0.0862,NaN
LIVINGAPARTMENTS_MODE,210199,0.6835,18122,0.0862,97312,6703,0.0862,NaN
LIVINGAPARTMENTS_AVG,210199,0.6835,18122,0.0862,97312,6703,0.0862,NaN


'display.max_columns'

The goal of creating the above summary of all 'NaN' entries in the main data table was to investigate two specific questions concerning these 67 features that each have at least one 'NaN' entry:

1. Whether simply having 'NaN' for a particular feature could help predict whether a borrower was delinquent.
2. Whether certain features with numerous 'NaN' entries would even be useful in predicting whether a borrower was delinquent. I am suspicious that a few of these features, such as `COMMONAREA_MEDI`, may have too few non-'NaN' entries that belong to delinquent payers. If too small a portion of the training set's target segment is included in these features, it's hard to see how they will be useful in making predictions that generalize to unseen datapoints.

In the case of question `1.`, I can confirm that for all 67 features, a borrower simply having an 'NaN' entry for a feature does not in any way shape or form predict whether the borrower will be more or less likely to be delinquent. I verified this for each feature by looking at the fractions of both its 'NaN' and non-'NaN' cohorts that were delinquents. For each feature, I found that the fraction of 'NaN' borrowers who were delinquent was identical to the fraction of non-'NaN' borrowers who were delinquent. If being 'NaN' for a particular feature were to have any chance of being a meaningful predictor of delinquency, I would have expected these two proportions to have had a stastically significant difference for that feature.

For question `2.`, it's helpful to remember that, as confirmed above, there are 24,825 borrowers in the training dataset who were delinquent repayers (who had a `TARGET` value of 1). Any feature that I retain in spite of its 'NaN' entries needs to still have a large enough amount of non-'NaN' entries that belong to delinquents. If the entire training set's delinquent population is not adequately represented among a particular feature's valid data points, it's unlikely that any predictions meaningfully informed by this feature would generalize well to unseen datapoints. The effective training set size for this feature would be just too small, and my model would be at risk of underfitting.

However, the big question is: what should the cutoff line be? What fraction of the 24,825 delinquent borrowers in the training set need to be captured by a feature's valid data points in order for the feature to have a chance of being useful in making predictions? There is no general rule of thumb that I can use to answer this for each of these features that has 'NaN' values. The short answer is, it depends -- on many factors such as distribution of the feature's valid data, as well as the type of classifier algorithm I'm using. 

This is why for the time being I won't remove *any* of these features -- even features like `COMMONAREA_MEDI` that contain 'NaN' in over two-thirds of their entries. Instead of running detailed statistical analyses on these features' distributions, I will instead experiment with dimensionality reduction algorithms such as PCA and/or feature selection algorithms such as SelectKBest.

If a feature had had, say, 95% of its entries as 'NaN', I probably would have removed it from the dataset at this point. However, since the sparsest features in the main data table still have valid data in just over 30% of their entries, I don't want to prematurely remove a feature that may have a chance, however remote, of possibly contributing to useful predictions.

### Main Data Table Featureset Exploration

1. **SK_ID_CURR**: ID of loan in our sample	
* **TARGET**: Target variable (1 - client with payment difficulties: he/she had late payment more than X days on at least one of the first Y installments of the loan in our sample, 0 - all other cases)	
* **NAME_CONTRACT_TYPE**: Identification if loan is cash or revolving	
* **CODE_GENDER**: Gender of the client	
* **FLAG_OWN_CAR**: Flag if the client owns a car	
* **FLAG_OWN_REALTY**: Flag if client owns a house or flat	
* **CNT_CHILDREN**: Number of children the client has	
* **AMT_INCOME_TOTAL**: Income of the client	
* **AMT_CREDIT**: Credit amount of the loan	
* **AMT_ANNUITY**: Loan annuity	
* **AMT_GOODS_PRICE**: For consumer loans it is the price of the goods for which the loan is given	
* **NAME_TYPE_SUITE**: Who was accompanying client when he was applying for the loan	
* **NAME_INCOME_TYPE**: Clients income type (businessman, working, maternity leave,Ö)	
* **NAME_EDUCATION_TYPE**: Level of highest education the client achieved
* **NAME_FAMILY_STATUS**: Family status of the client	
* **NAME_HOUSING_TYPE**: What is the housing situation of the client (renting, living with parents, ...)	
* **REGION_POPULATION_RELATIVE**: Normalized population of region where client lives (higher number means the client lives in more populated region) -- normalized
* **DAYS_BIRTH**: Client's age in days at the time of application -- time only relative to the application
* **DAYS_EMPLOYED**: How many days before the application the person started current employment -- time only relative to the application
* **DAYS_REGISTRATION**: How many days before the application did client change his registration -- time only relative to the application
* **DAYS_ID_PUBLISH**: How many days before the application did client change the identity document with which he applied for the loan -- time only relative to the application
* **OWN_CAR_AGE**: Age of client's car	
* **FLAG_MOBIL**: Did client provide mobile phone (1=YES, 0=NO)	
* **FLAG_EMP_PHONE**: Did client provide work phone (1=YES, 0=NO)	
* **FLAG_WORK_PHONE**: Did client provide home phone (1=YES, 0=NO)	
* **FLAG_CONT_MOBILE**: Was mobile phone reachable (1=YES, 0=NO)	
* **FLAG_PHONE**: Did client provide home phone (1=YES, 0=NO)	
* **FLAG_EMAIL**: Did client provide email (1=YES, 0=NO)	
* **OCCUPATION_TYPE**: What kind of occupation does the client have	
* **CNT_FAM_MEMBERS**: How many family members does client have	
* **REGION_RATING_CLIENT**: Our rating of the region where client lives (1,2,3)	
* **REGION_RATING_CLIENT_W_CITY**: Our rating of the region where client lives with taking city into account (1,2,3)	
* **WEEKDAY_APPR_PROCESS_START**: On which day of the week did the client apply for the loan	
* **HOUR_APPR_PROCESS_START**: Approximately at what hour did the client apply for the loan	rounded
* **REG_REGION_NOT_LIVE_REGION**: Flag if client's permanent address does not match contact address (1=different, 0=same, at region level)	
* **REG_REGION_NOT_WORK_REGION**: Flag if client's permanent address does not match work address (1=different, 0=same, at region level)	
* **LIVE_REGION_NOT_WORK_REGION**: Flag if client's contact address does not match work address (1=different, 0=same, at region level)	
* **REG_CITY_NOT_LIVE_CITY**: Flag if client's permanent address does not match contact address (1=different, 0=same, at city level)	
* **REG_CITY_NOT_WORK_CITY**: Flag if client's permanent address does not match work address (1=different, 0=same, at city level)	
* **LIVE_CITY_NOT_WORK_CITY**: Flag if client's contact address does not match work address (1=different, 0=same, at city level)	
* **ORGANIZATION_TYPE**: Type of organization where client works	
* **EXT_SOURCE_1**: Normalized score from external data source -- normalized
* **EXT_SOURCE_2**: Normalized score from external data source -- normalized
* **EXT_SOURCE_3**: Normalized score from external data source -- normalized
* **APARTMENTS_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **BASEMENTAREA_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **YEARS_BEGINEXPLUATATION_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **YEARS_BUILD_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **COMMONAREA_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **ELEVATORS_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **ENTRANCES_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **FLOORSMAX_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **FLOORSMIN_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **LANDAREA_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **LIVINGAPARTMENTS_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **LIVINGAREA_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **NONLIVINGAPARTMENTS_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **NONLIVINGAREA_AVG**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **APARTMENTS_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **BASEMENTAREA_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **YEARS_BEGINEXPLUATATION_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **YEARS_BUILD_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **COMMONAREA_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **ELEVATORS_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **ENTRANCES_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **FLOORSMAX_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **FLOORSMIN_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **LANDAREA_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **LIVINGAPARTMENTS_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **LIVINGAREA_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **NONLIVINGAPARTMENTS_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **NONLIVINGAREA_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **APARTMENTS_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **BASEMENTAREA_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **YEARS_BEGINEXPLUATATION_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **YEARS_BUILD_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **COMMONAREA_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **ELEVATORS_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **ENTRANCES_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **FLOORSMAX_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **FLOORSMIN_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **LANDAREA_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **LIVINGAPARTMENTS_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **LIVINGAREA_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **NONLIVINGAPARTMENTS_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **NONLIVINGAREA_MEDI**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **FONDKAPREMONT_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **HOUSETYPE_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **TOTALAREA_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **WALLSMATERIAL_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **EMERGENCYSTATE_MODE**: Normalized information about building where the client lives, What is average (_AVG suffix), modus (_MODE suffix), median (_MEDI suffix) apartment size, common area, living area, age of building, number of elevators, number of entrances, state of the building, number of floor -- normalized
* **OBS_30_CNT_SOCIAL_CIRCLE**: How many observation of client's social surroundings with observable 30 DPD (days past due) default	
* **DEF_30_CNT_SOCIAL_CIRCLE**: How many observation of client's social surroundings defaulted on 30 DPD (days past due) 	
* **OBS_60_CNT_SOCIAL_CIRCLE**: How many observation of client's social surroundings with observable 60 DPD (days past due) default	
* **DEF_60_CNT_SOCIAL_CIRCLE**: How many observation of client's social surroundings defaulted on 60 (days past due) DPD	
* **DAYS_LAST_PHONE_CHANGE**: How many days before application did client change phone	
* **FLAG_DOCUMENT_2**: Did client provide document 2	
* **FLAG_DOCUMENT_3**: Did client provide document 3	
* **FLAG_DOCUMENT_4**: Did client provide document 4	
* **FLAG_DOCUMENT_5**: Did client provide document 5	
* **FLAG_DOCUMENT_6**: Did client provide document 6	
* **FLAG_DOCUMENT_7**: Did client provide document 7	
* **FLAG_DOCUMENT_8**: Did client provide document 8	
* **FLAG_DOCUMENT_9**: Did client provide document 9	
* **FLAG_DOCUMENT_10**: Did client provide document 10	
* **FLAG_DOCUMENT_11**: Did client provide document 11	
* **FLAG_DOCUMENT_12**: Did client provide document 12	
* **FLAG_DOCUMENT_13**: Did client provide document 13	
* **FLAG_DOCUMENT_14**: Did client provide document 14	
* **FLAG_DOCUMENT_15**: Did client provide document 15	
* **FLAG_DOCUMENT_16**: Did client provide document 16	
* **FLAG_DOCUMENT_17**: Did client provide document 17	
* **FLAG_DOCUMENT_18**: Did client provide document 18	
* **FLAG_DOCUMENT_19**: Did client provide document 19	
* **FLAG_DOCUMENT_20**: Did client provide document 20	
* **FLAG_DOCUMENT_21**: Did client provide document 21	
* **AMT_REQ_CREDIT_BUREAU_HOUR**: Number of enquiries to Credit Bureau about the client one hour before application
* **AMT_REQ_CREDIT_BUREAU_DAY**: Number of enquiries to Credit Bureau about the client one day before application (excluding one hour before application)	
* **AMT_REQ_CREDIT_BUREAU_WEEK**: Number of enquiries to Credit Bureau about the client one week before application (excluding one day before application)	
* **AMT_REQ_CREDIT_BUREAU_MON**: Number of enquiries to Credit Bureau about the client one month before application (excluding one week before application)	
* **AMT_REQ_CREDIT_BUREAU_QRT**: Number of enquiries to Credit Bureau about the client 3 month before application (excluding one month before application)	
* **AMT_REQ_CREDIT_BUREAU_YEAR**: Number of enquiries to Credit Bureau about the client one day year (excluding last 3 months before application)

### 2. Bureau Data Table (bureau_balance.csv)

In [25]:
# Display the first five records
display(bureau_data.head(n=5))

,SK_ID_CURR,SK_ID_BUREAU,CREDIT_ACTIVE,CREDIT_CURRENCY,DAYS_CREDIT,CREDIT_DAY_OVERDUE,DAYS_CREDIT_ENDDATE,DAYS_ENDDATE_FACT,AMT_CREDIT_MAX_OVERDUE,CNT_CREDIT_PROLONG,AMT_CREDIT_SUM,AMT_CREDIT_SUM_DEBT,AMT_CREDIT_SUM_LIMIT,AMT_CREDIT_SUM_OVERDUE,CREDIT_TYPE,DAYS_CREDIT_UPDATE,AMT_ANNUITY
0,215354,5714462,Closed,currency 1,-497,0,-153.0,-153.0,NaN,0,91323.0,0.0,NaN,0.0,Consumer credit,-131,NaN
1,215354,5714463,Active,currency 1,-208,0,1075.0,NaN,NaN,0,225000.0,171342.0,NaN,0.0,Credit card,-20,NaN
2,215354,5714464,Active,currency 1,-203,0,528.0,NaN,NaN,0,464323.5,NaN,NaN,0.0,Consumer credit,-16,NaN
3,215354,5714465,Active,currency 1,-203,0,NaN,NaN,NaN,0,90000.0,NaN,NaN,0.0,Credit card,-16,NaN
4,215354,5714466,Active,currency 1,-629,0,1197.0,NaN,77674.5,0,2700000.0,NaN,NaN,0.0,Consumer credit,-21,NaN


**Bureau Data Table Featureset Exploration**

1. **SK_ID_CURR**: ID of loan in our sample - one loan in our sample can have 0,1,2 or more related previous credits in credit bureau -- hashed
* **SK_BUREAU_ID**: Recoded ID of previous Credit Bureau credit related to our loan (unique coding for each loan application) -- hashed
* **CREDIT_ACTIVE**: Status of the Credit Bureau (CB) reported credits	
* **CREDIT_CURRENCY**: Recoded currency of the Credit Bureau credit -- recoded
* **DAYS_CREDIT**: How many days before current application did client apply for Credit Bureau credit -- time only relative to the application
* **CREDIT_DAY_OVERDUE**: Number of days past due on CB credit at the time of application for related loan in our sample	
* **DAYS_CREDIT_ENDDATE**: Remaining duration of CB credit (in days) at the time of application in Home Credit -- time only relative to the application
* **DAYS_ENDDATE_FACT**: Days since CB credit ended at the time of application in Home Credit (only for closed credit) -- time only relative to the application
* **AMT_CREDIT_MAX_OVERDUE**: Maximal amount overdue on the Credit Bureau credit so far (at application date of loan in our sample)	
* **CNT_CREDIT_PROLONG**: How many times was the Credit Bureau credit prolonged	
* **AMT_CREDIT_SUM**: Current credit amount for the Credit Bureau credit
* **AMT_CREDIT_SUM_DEBT**: Current debt on Credit Bureau credit	
* **AMT_CREDIT_SUM_LIMIT**: Current credit limit of credit card reported in Credit Bureau	
* **AMT_CREDIT_SUM_OVERDUE**: Current amount overdue on Credit Bureau credit	
* **CREDIT_TYPE**: Type of Credit Bureau credit (Car, cash,...)	
* **DAYS_CREDIT_UPDATE**: How many days before loan application did last information about the Credit Bureau credit come -- time only relative to the application
* **AMT_ANNUITY**: Annuity of the Credit Bureau credit	

### 3. Bureau Balance Data Table (bureau_balance.csv)

In [24]:
# Display the first five records
display(bureau_balance_data.head(n=5))

,SK_ID_BUREAU,MONTHS_BALANCE,STATUS
0,5715448,0,C
1,5715448,-1,C
2,5715448,-2,C
3,5715448,-3,C
4,5715448,-4,C


**Bureau Balance Data Table Featureset Exploration**

1. **SK_BUREAU_ID**: Recoded ID of Credit Bureau credit (unique coding for each application) - use this to join to CREDIT_BUREAU table -- hashed
* **MONTHS_BALANCE**: Month of balance relative to application date (-1 means the freshest balance date) -- time only relative to the application
* **STATUS**: Status of Credit Bureau loan during the month (active, closed, DPD0-30,Ö [C means closed, X means status unknown, 0 means no DPD, 1 means maximal did during month between 1-30, 2 means DPD 31-60,Ö 5 means DPD 120+ or sold or written off ])

### 4. Previous Application Data Table (previous_application.csv)

In [26]:
# Display the first five records
display(previous_application_data.head(n=5))

,SK_ID_PREV,SK_ID_CURR,NAME_CONTRACT_TYPE,AMT_ANNUITY,AMT_APPLICATION,AMT_CREDIT,AMT_DOWN_PAYMENT,AMT_GOODS_PRICE,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,...,NAME_SELLER_INDUSTRY,CNT_PAYMENT,NAME_YIELD_GROUP,PRODUCT_COMBINATION,DAYS_FIRST_DRAWING,DAYS_FIRST_DUE,DAYS_LAST_DUE_1ST_VERSION,DAYS_LAST_DUE,DAYS_TERMINATION,NFLAG_INSURED_ON_APPROVAL
0,2030495,271877,Consumer loans,1730.430,17145.0,17145.0,0.0,17145.0,SATURDAY,15,...,Connectivity,12.0,middle,POS mobile with interest,365243.0,-42.0,300.0,-42.0,-37.0,0.0
1,2802425,108129,Cash loans,25188.615,607500.0,679671.0,NaN,607500.0,THURSDAY,11,...,XNA,36.0,low_action,Cash X-Sell: low,365243.0,-134.0,916.0,365243.0,365243.0,1.0
2,2523466,122040,Cash loans,15060.735,112500.0,136444.5,NaN,112500.0,TUESDAY,11,...,XNA,12.0,high,Cash X-Sell: high,365243.0,-271.0,59.0,365243.0,365243.0,1.0
3,2819243,176158,Cash loans,47041.335,450000.0,470790.0,NaN,450000.0,MONDAY,7,...,XNA,12.0,middle,Cash X-Sell: middle,365243.0,-482.0,-152.0,-182.0,-177.0,1.0
4,1784265,202054,Cash loans,31924.395,337500.0,404055.0,NaN,337500.0,THURSDAY,9,...,XNA,24.0,high,Cash Street: high,NaN,NaN,NaN,NaN,NaN,NaN


**Previous Application Data Table Featureset Exploration**

1. **SK_ID_PREV**: ID of previous credit in Home credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loan applications in Home Credit, previous application could, but not necessarily have to lead to credit) -- hashed
* **SK_ID_CURR**: ID of loan in our sample -- hashed
* **NAME_CONTRACT_TYPE**: Contract product type (Cash loan, consumer loan [POS] ,...) of the previous application	
* **AMT_ANNUITY**: Annuity of previous application	
* **AMT_APPLICATION**: For how much credit did client ask on the previous application	
* **AMT_CREDIT**: Final credit amount on the previous application. This differs from AMT_APPLICATION in a way that the AMT_APPLICATION is the amount for which the client initially applied for, but during our approval process he could have received different amount - AMT_CREDIT	
* **AMT_DOWN_PAYMENT**: Down payment on the previous application	
* **AMT_GOODS_PRICE**: Goods price of good that client asked for (if applicable) on the previous application	
* **WEEKDAY_APPR_PROCESS_START**: On which day of the week did the client apply for previous application	
* **HOUR_APPR_PROCESS_START**: Approximately at what day hour did the client apply for the previous application -- rounded
* **FLAG_LAST_APPL_PER_CONTRACT**: Flag if it was last application for the previous contract. Sometimes by mistake of client or our clerk there could be more applications for one single contract	
* **NFLAG_LAST_APPL_IN_DAY**: Flag if the application was the last application per day of the client. Sometimes clients apply for more applications a day. Rarely it could also be error in our system that one application is in the database twice	
* **RATE_DOWN_PAYMENT**: Down payment rate normalized on previous credit -- normalized
* **RATE_INTEREST_PRIMARY**: Interest rate normalized on previous credit -- normalized
* **RATE_INTEREST_PRIVILEGED**: Interest rate normalized on previous credit -- normalized
* **NAME_CASH_LOAN_PURPOSE**: Purpose of the cash loan	
* **NAME_CONTRACT_STATUS**: Contract status (approved, cancelled, ...) of previous application	
* **DAYS_DECISION**: Relative to current application when was the decision about previous application made	time only relative to the application
* **NAME_PAYMENT_TYPE**: Payment method that client chose to pay for the previous application	
* **CODE_REJECT_REASON**: Why was the previous application rejected	
* **NAME_TYPE_SUITE**: Who accompanied client when applying for the previous application	
* **NAME_CLIENT_TYPE**: Was the client old or new client when applying for the previous application	
* **NAME_GOODS_CATEGORY**: What kind of goods did the client apply for in the previous application	
* **NAME_PORTFOLIO**: Was the previous application for CASH, POS, CAR, Ö
* **NAME_PRODUCT_TYPE**: Was the previous application x-sell o walk-in	
* **CHANNEL_TYPE**: Through which channel we acquired the client on the previous application	
* **SELLERPLACE_AREA**: Selling area of seller place of the previous application	
* **NAME_SELLER_INDUSTRY**: The industry of the seller	
* **CNT_PAYMENT**: Term of previous credit at application of the previous application	
* **NAME_YIELD_GROUP**: Grouped interest rate into small medium and high of the previous application -- grouped
* **PRODUCT_COMBINATION**: Detailed product combination of the previous application	
* **DAYS_FIRST_DRAWING**: Relative to application date of current application when was the first disbursement of the previous application -- time only relative to the application
* **DAYS_FIRST_DUE**: Relative to application date of current application when was the first due supposed to be of the previous application -- time only relative to the application
* **DAYS_LAST_DUE_1ST_VERSION**: Relative to application date of current application when was the first due of the previous application -- time only relative to the application
* **DAYS_LAST_DUE**: Relative to application date of current application when was the last due date of the previous application -- time only relative to the application
* **DAYS_TERMINATION**: Relative to application date of current application when was the expected termination of the previous application -- time only relative to the application
* **NFLAG_INSURED_ON_APPROVAL**: Did the client requested insurance during the previous application

### 5. POS CASH Balance Data Table (POS_CASH_balance.csv)

In [20]:
# Display the first five records
display(POS_CASH_balance_data.head(n=5))

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,CNT_INSTALMENT,CNT_INSTALMENT_FUTURE,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,1803195,182943,-31,48.0,45.0,Active,0,0
1,1715348,367990,-33,36.0,35.0,Active,0,0
2,1784872,397406,-32,12.0,9.0,Active,0,0
3,1903291,269225,-35,48.0,42.0,Active,0,0
4,2341044,334279,-35,36.0,35.0,Active,0,0


**POS CASH Balance Data Table Featureset Exploration**

1. **SK_ID_PREV**: ID of previous credit in Home Credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loans in Home Credit)	
* **SK_ID_CURR**: ID of loan in our sample	
* **MONTHS_BALANCE**: Month of balance relative to application date (-1 means the information to the freshest monthly snapshot, 0 means the information at application - often it will be the same as -1 as many banks are not updating the information to Credit Bureau regularly ) -- time only relative to the application
* **CNT_INSTALMENT**: Term of previous credit (can change over time)	
* **CNT_INSTALMENT_FUTURE**: Installments left to pay on the previous credit	
* **NAME_CONTRACT_STATUS**: Contract status during the month	
* **SK_DPD**: DPD (days past due) during the month of previous credit	
* **SK_DPD_DEF**: DPD during the month with tolerance (debts with low loan amounts are ignored) of the previous credit

### 6. Installments Payments Data Table (installments_payments.csv)

In [21]:
# Display the first five records
display(installments_payments_data.head(n=5))

,SK_ID_PREV,SK_ID_CURR,NUM_INSTALMENT_VERSION,NUM_INSTALMENT_NUMBER,DAYS_INSTALMENT,DAYS_ENTRY_PAYMENT,AMT_INSTALMENT,AMT_PAYMENT
0,1054186,161674,1.0,6,-1180.0,-1187.0,6948.360,6948.360
1,1330831,151639,0.0,34,-2156.0,-2156.0,1716.525,1716.525
2,2085231,193053,2.0,1,-63.0,-63.0,25425.000,25425.000
3,2452527,199697,1.0,3,-2418.0,-2426.0,24350.130,24350.130
4,2714724,167756,1.0,2,-1383.0,-1366.0,2165.040,2160.585


**Installments Payments Data Table Featureset Exploration**

1. **SK_ID_PREV**: ID of previous credit in Home credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loans in Home Credit) -- hashed
* **SK_ID_CURR**: ID of loan in our sample -- hashed
* **NUM_INSTALMENT_VERSION**: Version of installment calendar (0 is for credit card) of previous credit. Change of installment version from month to month signifies that some parameter of payment calendar has changed	
* **NUM_INSTALMENT_NUMBER**: On which installment we observe payment	
* **DAYS_INSTALMENT**: When the installment of previous credit was supposed to be paid (relative to application date of current loan) -- time only relative to the application
* **DAYS_ENTRY_PAYMENT**: When was the installments of previous credit paid actually (relative to application date of current loan) -- time only relative to the application
* **AMT_INSTALMENT**: What was the prescribed installment amount of previous credit on this installment	
* **AMT_PAYMENT**: What the client actually paid on previous credit on this installment

### 7. Credit Card Balance Data Table (credit_card_balance.csv)

In [22]:
# Display the first five records
display(credit_card_balance_data.head(n=5))

,SK_ID_PREV,SK_ID_CURR,MONTHS_BALANCE,AMT_BALANCE,AMT_CREDIT_LIMIT_ACTUAL,AMT_DRAWINGS_ATM_CURRENT,AMT_DRAWINGS_CURRENT,AMT_DRAWINGS_OTHER_CURRENT,AMT_DRAWINGS_POS_CURRENT,AMT_INST_MIN_REGULARITY,...,AMT_RECIVABLE,AMT_TOTAL_RECEIVABLE,CNT_DRAWINGS_ATM_CURRENT,CNT_DRAWINGS_CURRENT,CNT_DRAWINGS_OTHER_CURRENT,CNT_DRAWINGS_POS_CURRENT,CNT_INSTALMENT_MATURE_CUM,NAME_CONTRACT_STATUS,SK_DPD,SK_DPD_DEF
0,2562384,378907,-6,56.970,135000,0.0,877.5,0.0,877.5,1700.325,...,0.000,0.000,0.0,1,0.0,1.0,35.0,Active,0,0
1,2582071,363914,-1,63975.555,45000,2250.0,2250.0,0.0,0.0,2250.000,...,64875.555,64875.555,1.0,1,0.0,0.0,69.0,Active,0,0
2,1740877,371185,-7,31815.225,450000,0.0,0.0,0.0,0.0,2250.000,...,31460.085,31460.085,0.0,0,0.0,0.0,30.0,Active,0,0
3,1389973,337855,-4,236572.110,225000,2250.0,2250.0,0.0,0.0,11795.760,...,233048.970,233048.970,1.0,1,0.0,0.0,10.0,Active,0,0
4,1891521,126868,-1,453919.455,450000,0.0,11547.0,0.0,11547.0,22924.890,...,453919.455,453919.455,0.0,1,0.0,1.0,101.0,Active,0,0


**Credit Card Balance Data Table Featureset Exploration**

1. **SK_ID_PREV**: ID of previous credit in Home credit related to loan in our sample. (One loan in our sample can have 0,1,2 or more previous loans in Home Credit) -- hashed
* **SK_ID_CURR**: ID of loan in our sample -- hashed
* **MONTHS_BALANCE**: Month of balance relative to application date (-1 means the freshest balance date) -- time only relative to the application
* **AMT_BALANCE**: Balance during the month of previous credit	
* **AMT_CREDIT_LIMIT_ACTUAL**: Credit card limit during the month of the previous credit	
* **AMT_DRAWINGS_ATM_CURRENT**: Amount drawing at ATM during the month of the previous credit	
* **AMT_DRAWINGS_CURRENT**: Amount drawing during the month of the previous credit	
* **AMT_DRAWINGS_OTHER_CURRENT**: Amount of other drawings during the month of the previous credit	
* **AMT_DRAWINGS_POS_CURRENT**: Amount drawing or buying goods during the month of the previous credit	
* **AMT_INST_MIN_REGULARITY**: Minimal installment for this month of the previous credit	
* **AMT_PAYMENT_CURRENT**: How much did the client pay during the month on the previous credit	
* **AMT_PAYMENT_TOTAL_CURRENT**: How much did the client pay during the month in total on the previous credit	
* **AMT_RECEIVABLE_PRINCIPAL**: Amount receivable for principal on the previous credit	
* **AMT_RECIVABLE**: Amount receivable on the previous credit	
* **AMT_TOTAL_RECEIVABLE**: Total amount receivable on the previous credit
* **CNT_DRAWINGS_ATM_CURRENT**: Number of drawings at ATM during this month on the previous credit	
* **CNT_DRAWINGS_CURRENT**: Number of drawings during this month on the previous credit	
* **CNT_DRAWINGS_OTHER_CURRENT**: Number of other drawings during this month on the previous credit	
* **CNT_DRAWINGS_POS_CURRENT**: Number of drawings for goods during this month on the previous credit	
* **CNT_INSTALMENT_MATURE_CUM**: Number of paid installments on the previous credit	
* **NAME_CONTRACT_STATUS**: Contract status (active signed,...) on the previous credit	
* **SK_DPD**: DPD (Days past due) during the month on the previous credit
* **SK_DPD_DEF**: DPD (Days past due) during the month with tolerance (debts with low loan amounts are ignored) of the previous credit